## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

170500096/170498071 [==============================] - 28s 0us/step
(50000, 32, 32, 3)


In [3]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

In [6]:

classifier=Sequential()

#卷積組合
classifier.add(Convolution2D(filters = 32, kernel_size = (3, 3), input_shape = (32, 32, 3), activation = 'relu'))#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(BatchNormalization())

'''自己決定MaxPooling2D放在哪裡'''
classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合
classifier.add(Convolution2D(filters = 64, kernel_size = (3, 3), activation = 'relu'))
classifier.add(BatchNormalization())

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(100, activation = 'relu')) #output_dim=100,activation=relu

#輸出
classifier.add(Dense(10,activation='softmax'))

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=100,epochs=10)

Epoch 1/10
500/500 [==============================] - 165s 331ms/step - loss: 1.3746 - accuracy: 0.5508
Epoch 2/10
500/500 [==============================] - 180s 360ms/step - loss: 0.7747 - accuracy: 0.7296
Epoch 3/10
500/500 [==============================] - 173s 347ms/step - loss: 0.5158 - accuracy: 0.8196
Epoch 4/10
500/500 [==============================] - 194s 387ms/step - loss: 0.3206 - accuracy: 0.8894
Epoch 5/10
500/500 [==============================] - 191s 381ms/step - loss: 0.1946 - accuracy: 0.9331
Epoch 6/10
500/500 [==============================] - 189s 379ms/step - loss: 0.1453 - accuracy: 0.9499
Epoch 7/10
500/500 [==============================] - 167s 335ms/step - loss: 0.1286 - accuracy: 0.9554
Epoch 8/10
500/500 [==============================] - 185s 370ms/step - loss: 0.1114 - accuracy: 0.9627
Epoch 9/10
500/500 [==============================] - 206s 412ms/step - loss: 0.1003 - accuracy: 0.9657
Epoch 10/10
500/500 [==============================] - 220s 440m

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [7]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[2.6262966e-01, 2.7036229e-13, 7.3672181e-01, 2.4576391e-06,
        9.0709271e-09, 5.0426312e-16, 1.3290906e-12, 3.2896453e-11,
        6.4606575e-04, 2.9635928e-14]], dtype=float32)